In [1]:
from calvin_utils.dcm_utils.dcm_to_bids import organize_dcm_to_bids
organize_dcm_to_bids(src_root  = '/Volumes/LaCie/rolston', 
                     bids_root = '/Volumes/LaCie/BIDS', 
                     dcm_pattern_list=['3D_MPRAGE'])

/Volumes/LaCie/rolston/54456298/E54688428/MR
/Volumes/LaCie/rolston/54456298/E56826373/PR
/Volumes/LaCie/rolston/54456298/E56826373/MR
/Volumes/LaCie/rolston/54456298/E54688428/MR/AAHead_Scout_MPR_cor
/Volumes/LaCie/rolston/54456298/E54688428/MR/Ax_T2
/Volumes/LaCie/rolston/54456298/E54688428/MR/Post_Ax_T1_3D_MPRAGE
/Volumes/LaCie/rolston/54456298/E54688428/MR/Ax_T2_FLAIR
/Volumes/LaCie/rolston/54456298/E54688428/MR/Pha_Images
/Volumes/LaCie/rolston/54456298/E54688428/MR/Mag_Images
/Volumes/LaCie/rolston/54456298/E54688428/MR/Ax_DWI_ADC
/Volumes/LaCie/rolston/54456298/E54688428/MR/AAHead_Scout_MPR_tra
/Volumes/LaCie/rolston/54456298/E54688428/MR/Ax_DWI_TRACEW
/Volumes/LaCie/rolston/54456298/E54688428/MR/AAHead_Scout_MPR_sag
/Volumes/LaCie/rolston/54456298/E54688428/MR/Post_Ax_T1_3D_M_GE_MPR_Cor
/Volumes/LaCie/rolston/54456298/E54688428/MR/Post_Ax_T1_3D_M_GE_MPR_Sag
/Volumes/LaCie/rolston/54456298/E54688428/MR/Pre_Ax_T1_3D_MP_GE_MPR_Cor
/Volumes/LaCie/rolston/54456298/E54688428/MR/Pre_A